In [41]:
# 라이브러리 호출
import requests
import json
from bs4 import BeautifulSoup as bts
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import os

In [43]:
# 네이버뉴스 링크를 수집하고 데이터프레임으로 반환하는 함수 생성
def NaverNewsLink(searchWord, startNo = 1):
    
    # HTTP 요청 실행
    # 쿼리 문자열: 정렬(sort)-관련도순, 기간(pd)-전체
    res = requests.get(
        url = 'https://s.search.naver.com/p/newssearch/search.naver',         
        params = {
            'nqx_theme': {'theme':{'main':{'name':'sports_event'},'sub':[{'name':'issue'}]}},
            'query': searchWord,
            'sort': 0, #관련도순:0, 최신순:1, 오래된순:2
            'spq': 3,
            'pd': 0,
            'start': startNo,
            'where': 'news_tab_api',
        }
    )
    
    # JSON 형태의 문자열을 딕셔너리로 변환
    dic = json.loads(s = res.text)
    
    # 딕셔너리의 각 원소를 bs4.BeautifulSoup 객체로 변환
    # [주의] HTTP 응답 바디 문자열에 공백이 있으면 items의 원소로 추가되므로 삭제해야 함
    items = [bts(markup = i.strip(), features = 'html.parser') for i in dic['contents']]
    
    # 뉴스 링크를 포함하는 HTML 요소를 원소로 갖는 links 생성
    links = [item.select('a.info:last-child') for item in items]

    # 기사 작성 일시를 포함하는 HTML 요소를 원소로 갖는 dates 생성
    dates = [item.select('span.info')[0].text for item in items if len(item.select('span.info')) > 0]
    
    # 언론사, 제목 및 네이버뉴스 링크를 원소로 갖는 데이터프레임 생성
    newsList = pd.DataFrame(data = {
        'press': [item.select(selector = 'a.press')[0].text for item in items], 
        'title': [item.select(selector = 'a.news_tit')[0].text for item in items], 
        'nlink': [link[0]['href'] if len(link) >= 1 else np.nan for link in links],
        'date': dates
    })
    
    # 데이터프레임 반환
    return newsList

In [44]:
# 검색어 설정
searchWord = 'LG에너지솔루션'

In [45]:
# 함수 테스트
NaverNewsLink(searchWord = searchWord)

,press,title,nlink,date
0,한국경제언론사 선정,"이번엔 '13조' 터졌다…LG엔솔, 일주일 만에 또 '잭팟'",https://n.news.naver.com/mnews/article/015/000...,A13면 1단
1,매일경제언론사 선정,K배터리 제대로 터졌다…美 포드와 13조 규모 잭팟 계약 맺은 LG엔솔,https://n.news.naver.com/mnews/article/009/000...,2일 전
2,세계일보언론사 선정,"LG엔솔, 美 포드에 2032년까지 배터리 공급",https://n.news.naver.com/mnews/article/022/000...,A17면 1단
3,뉴스1,"배터리 넘어 에너지순환 생태계 선도…LG엔솔 ""2028년 매출 2배""",https://n.news.naver.com/mnews/article/421/000...,35분 전
4,뉴시스언론사 선정,LFP부터 46파이 원통형까지…'잭팟' 배터리 장점은[LG엔솔 달린다②],https://n.news.naver.com/mnews/article/003/001...,1일 전
5,한경비즈니스언론사 선정,"LG엔솔, 포드와 배터리 동맹…'13조 잭팟' 터졌다",https://n.news.naver.com/mnews/article/050/000...,2일 전
6,노컷뉴스,"LG에너지솔루션 '함솔이' 봉사단, 독거어르신 생신 잔치 봉사",https://n.news.naver.com/mnews/article/079/000...,1일 전
7,경향신문언론사 선정,[단독]LG엔솔 ‘불법파견’ 의혹…배터리 생산 공정 하청 노동자 4명 소...,https://n.news.naver.com/mnews/article/032/000...,1일 전
8,전자신문언론사 선정,"LG엔솔, 포드 상용차에 배터리 공급…13조 이상 초대형 수주",https://n.news.naver.com/mnews/article/030/000...,17면 TOP
9,스포츠조선,"LG에너지솔루션, 포드에 상용차용 배터리 공급",https://n.news.naver.com/mnews/article/076/000...,2일 전


In [46]:
# 수집 건수 설정
count = 1000

In [51]:
# 최종 결과를 저장할 빈 데이터프레임 생성
newsList = pd.DataFrame()

# 반복문으로 관련 네이버뉴스 링크 수집
for i in tqdm(range(1, count + 1, 10)):
    
    # 네이버뉴스 링크를 수집하고 df에 할당
    df = NaverNewsLink(searchWord = searchWord, startNo = i)
    
    # 최종 결과에 df에 추가(행이름 삭제)
    newsList = pd.concat(objs = [newsList, df], ignore_index = True)
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]

In [68]:
# nlink가 결측인 행 삭제
newsList = newsList.loc[newsList['nlink'].notna(), :]

In [69]:
# press에서 '언론사 선정' 삭제
newsList['press'] = newsList['press'].str.replace(pat = '언론사 선정', repl = '')

In [70]:
# newsList의 행 개수 확인
newsList.shape[0]

398

In [71]:
# nlink에서 쿼리 문자열 삭제
newsList['nlink'] = newsList['nlink'].str.replace(pat = r'(\?.+)', repl = '', regex = True)

In [75]:
# 두 데이터프레임의 인덱스를 정렬하여 일치시키는 방법도 있어:
dups = newsList.duplicated(subset='nlink').reset_index(drop=True)
newsList = newsList.reset_index(drop=True)
newsList = newsList.loc[~dups, :]


# nlink가 중복인 행이 있으면 해당 행을 삭제
#newsList = newsList.loc[~dups, :]

In [76]:
# newsList의 행 개수 확인
newsList.shape[0]

294

In [77]:
# nlink에서 쿼리 문자열 삭제
newsList['nlink'] = newsList['nlink'].str.replace(pat = r'(\?.+)', repl = '', regex = True)

In [78]:
# newsList의 행이름 초기화
newsList = newsList.reset_index(drop = True)

In [79]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
289,세계일보,"LG엔솔·삼성SDI, 전기상용차 배터리 선보여",https://n.news.naver.com/mnews/article/022/000...,A15면 1단
290,파이낸셜뉴스,"LG엔솔·삼성SDI ""차세대 배터리 솔루션 선도""",https://n.news.naver.com/mnews/article/014/000...,14면 1단
291,디지털타임스,"이달훈 LG엔솔 상무 ""셀 13만개 분해한 데이터 기반 `BMTS` 개발"" 자신감",https://n.news.naver.com/mnews/article/029/000...,3주 전
292,연합뉴스,"LG엔솔, CATL 제치고 중국 현지서 배터리 품질 부문 1위 차지",https://n.news.naver.com/mnews/article/001/001...,3주 전
293,헤럴드경제,“신재생 에너지가 전기 가장 많이 만드는 시대 온다” [헤럴드 기업포...,https://n.news.naver.com/mnews/article/016/000...,4면 TOP


In [80]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [81]:
# newsList를 pkl 파일로 저장
pd.to_pickle(obj = newsList, filepath_or_buffer = 'Naver_News_List_3.pkl')

In [82]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'LG에너지솔루션.ipynb',
 'naver_news_hinix.csv',
 'Naver_News_List.pkl',
 'Naver_News_List_2.pkl',
 'Naver_News_List_3.pkl',
 'naver_news_samsung.csv',
 'SK하이닉스크롤링.ipynb',
 'SK하이닉스크롤링step2.ipynb',
 'Untitled.ipynb',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']